In [2]:
!pip install torch

     -------------------------------------- 159.8/159.8 MB 2.4 MB/s eta 0:00:00
     ---------------------------------------- 5.7/5.7 MB 4.3 MB/s eta 0:00:00
     ---------------------------------------- 1.7/1.7 MB 4.3 MB/s eta 0:00:00
     -------------------------------------- 176.9/176.9 KB 3.5 MB/s eta 0:00:00
     -------------------------------------- 133.3/133.3 KB 3.8 MB/s eta 0:00:00
     -------------------------------------- 228.5/228.5 MB 2.5 MB/s eta 0:00:00
     ---------------------------------------- 3.5/3.5 MB 1.8 MB/s eta 0:00:00
     -------------------------------------- 286.4/286.4 KB 4.5 MB/s eta 0:00:00
     -------------------------------------- 536.2/536.2 KB 5.6 MB/s eta 0:00:00


You should consider upgrading via the 'C:\Users\danda\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [6]:
import torch
import torch.nn as nn
from torch.nn import functional as F





In [9]:
# !wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

In [10]:
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [11]:
batch_size = 128 # no. of independent sequences processes in parallel
block_size = 256 #length of random data size to be trained. (possibly also called context length)
max_iters = 5000
eval_interval = 500
learning_rate = 3e-4
device = 'cpu'
if torch.cuda.is_available():
    device = 'cuda'
elif torch.backends.mps.is_available():
    device = 'mps'
print(device)
eval_iters = 50
n_embd = 192
n_head = 6
n_layer = 6
dropout = 0.2
#---------

torch.manual_seed(1337)

cpu


In [12]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(vocab_size)

65


In [13]:
stoi = { ch:i for i,ch in enumerate(chars)}
itos = { i:ch for i,ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])

In [14]:
#encode full dataset
data = torch.tensor(encode(text), dtype=torch.long)

#split training and validation data
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

C:\Users\danda\AppData\Local\Temp\ipykernel_13824\953841872.py:2: UserWarning: Failed to initialize NumPy: No module named 'numpy' (Triggered internally at ..\torch\csrc\utils\tensor_numpy.cpp:84.)
  data = torch.tensor(encode(text), dtype=torch.long)


In [15]:
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i + block_size] for i in ix])
    y = torch.stack([data[i+1:i + block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            x, y = get_batch(split)
            logits, loss = model(x, y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [16]:
class Head(nn.Module):
    """ one head of self attention"""

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x) # (B,T,C)
        q = self.query(x) # (B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * C**-.05 # (B, T, C) @ (B, C , T) = (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x)
        out = wei @ v
        return out


class MultiHeadAttention(nn.Module):
    """ multiple head of sel attention in parallel"""

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out


class FeedForward(nn.Module):
    """ a simple linear layer followed by non-linearity"""

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout)
        )

    def forward(self, x):
        return self.net(x)


class Block(nn.Module):
    """ Transformer block: communication followed by computation"""

    def __init__(self, n_embd, n_head):
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedForward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x


class BigramLanguageModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # Final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensors of integers
        tok_embd = self.token_embedding_table(idx)  # (B,T,C)
        pos_embd = self.position_embedding_table(torch.arange(T, device=device))  # (T, C)
        x = tok_embd + pos_embd  # (B,T,C)
        x = self.blocks(x)
        x = self.ln_f(x)
        logits = self.lm_head(x)  # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)  # equivalent to target.view(-1)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idc is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=1) # (B, c)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

In [17]:
torch.cuda.empty_cache()
torch.cuda.amp.autocast(enabled = True)

model = BigramLanguageModel()
model = model.to(device)

print(sum(p.numel() for p in model.parameters())/1e6, 'M Parameters')

optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)

scaler = torch.cuda.amp.GradScaler()

for iter in range(max_iters):
    torch.cuda.empty_cache()

    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate loss
    logits, loss = model(xb, yb) # m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    #loss.backward()
    scaler.scale(loss).backward()
    scaler.step(optimizer)
    scaler.update()
    #optimizer.step()

c:\Users\danda\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\amp\autocast_mode.py:250: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn(


2.740289 M Parameters


c:\Users\danda\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\amp\grad_scaler.py:131: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn(


In [37]:
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(model.generate(context, max_new_tokens=500)[0].tolist()))


shall you gentleman these woes, odds you know he eass?

ROMEO:
I am many ordainment withal. And after it,
When thou canst make farth, talk not not one,
To do speak not where thou not: to't, but I say,
Tell the allague measure his ever.

FRIAR LAURENCE:
With what wise?

FRIAR LAURENCE:
Unrod is my good indeed, believe me the face of soul
That flower and one rhides upon your service.

BRAKENBURY:
What, my unman?

BENVOLIO:
He must be your willing?

MERCUTIO:
Call them depent it thither now child f


In [38]:
torch.save(model.state_dict(), "S19ShakespeareanGPT_model.pth")